In [11]:
import tensorflow as tf
from tensorflow.keras import layers

In [216]:
class ConvolutionalBlockWithPool(tf.keras.Model):

    def __init__(self, filters, kernel_size, stride, padding='valid', **kwargs):
        super(ConvolutionalBlockWithPool, self).__init__(**kwargs)
        
        self.filters = filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        self.conv = layers.Conv2D(
            filters=self.filters,
            kernel_size=self.kernel_size,
            strides=self.stride,
            padding=self.padding
        )

        self.relu = layers.ReLU()
        
        self.max_pool = tf.keras.layers.MaxPool2D(
            pool_size=(3, 3),
            strides=(2, 2),
            padding='valid'
        )

    def call(self, inputs, training = None):

        net = self.conv(inputs)
        net = self.relu(net)
        
        net = self.max_pool(net)

        return net

In [227]:
class ConvolutionalBlock(tf.keras.Model):

    def __init__(self, filters, kernel_size, stride, **kwargs):
        super(ConvolutionalBlock, self).__init__(**kwargs)
        
        self.filters = filters
        self.kernel_size = kernel_size
        self.stride = stride
        
        self.conv = layers.Conv2D(
            filters=self.filters,
            kernel_size=kernel_size,
            strides=stride,
            padding='same'
        )

        self.relu = layers.ReLU()

    def call(self, inputs, training = None):

        net = self.conv(inputs)
        net = self.relu(net)

        return net

In [242]:
class DenseBlock(tf.keras.Model):

    def __init__(self, num_units, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
        
        self.num_units = num_units
        
        self.dropout = layers.Dropout(rate=0.2)
        self.dense = layers.Dense(units=self.num_units)
        self.relu = layers.ReLU()

    def call(self, inputs, training = None):

        net = self.dropout(inputs)
        net = self.dense(net)
        net = self.relu(net)

        return net

In [243]:
class AlexNetArchitecture(tf.keras.Model):

    def __init__(self,**kwargs):

        super(AlexNetArchitecture, self).__init__(**kwargs)

        self.conv_1 = ConvolutionalBlockWithPool(filters=96, kernel_size=[11,11], stride=(4,4))
        self.conv_2 = ConvolutionalBlockWithPool(filters=256, kernel_size=[5,5], stride=(1,1),padding='same')
        self.conv_3 = ConvolutionalBlock(filters=384, kernel_size=[3,3], stride=(1,1))
        self.conv_4 = ConvolutionalBlock(filters=384, kernel_size=[3,3], stride=(1,1))
        self.conv_5 = ConvolutionalBlockWithPool(filters=256, kernel_size=[3,3], stride=(1,1),padding='same')
        
        self.flatten = layers.Flatten()
        self.dense_1 = DenseBlock(num_units=4096)
        self.dense_2 = DenseBlock(num_units=4096)
        self.dense_3 = layers.Dense(units=1000)
        
    def call(self, inputs, training=None):
        
        net = self.conv_1(inputs)
        net = self.conv_2(net)
        net = self.conv_3(net)
        net = self.conv_4(net)
        net = self.conv_5(net)
        
        net = self.flatten(net)
        
        net = self.dense_1(net)
        net = self.dense_2(net)
        net = self.dense_3(net)
        
        return net

In [244]:
inputs = tf.keras.layers.Input(shape=(227, 227, 3))
alexnet = AlexNetArchitecture()
outputs = alexnet(inputs, training = False)

In [245]:
alexnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolutional_block_with_poo (None, 27, 27, 96)        34944     
_________________________________________________________________
convolutional_block_with_poo (None, 13, 13, 256)       614656    
_________________________________________________________________
convolutional_block_164 (Con (None, 13, 13, 384)       885120    
_________________________________________________________________
convolutional_block_165 (Con (None, 13, 13, 384)       1327488   
_________________________________________________________________
convolutional_block_with_poo (None, 6, 6, 256)         884992    
_________________________________________________________________
flatten_16 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_block (DenseBlock)     (None, 4096)              37752832  
__________